# Measuring the effects of axonal adaptations on firing properties of Mitral Cells

The effects of axonal adaptations on mitral cell firing are not well studied. 

I wanted to design a realistic mitral cell with myelinated axonal domains to determine how changes and adaptations in these domains influence action potential initiation and propagation.

## Model design
- mitral cell model (big soma and dendrite, active channels on both)
- based on literature and my experimental data
- implemented and simulated using the Python 3.8 bindings NEURON 7.7

# AIS


- Na<sub>v</sub>1.6
- Na<sub>v</sub>1.2 
- K<sub>v</sub>
- passive leak channels 
- The AIS in cortex displays dramatic ion channel distribution changes along its length ([Hu et al. 2009; Figure 6](https://www.nature.com/articles/nn.2359))
    
<img src="img/Hu_Shu_2009_fig5.png" alt="Hu et al. 2009" width=500/>


```python

def simple_ais_channel_gradient(self, gmap):
    """temporary method. Uses a linear increase or decrease of channel density based on nseg. 
    Need to make it more realistic based on Hu et al. 2009 Figure 5"""
    a = np.linspace(gmap["min"], gmap["max"], self.ais.nseg)
    if gmap["reverse"] == True:
        return a[::-1]
    return a

```
Where `gmap` is a dictionary/map containing the keys `min`, `max`, and `reverse`. 

```python
nav16_gmap = {"min":0, "max":2500, "nseg":15}

```
This function is used to generate a range of numbers representing the conductance changes along the AIS from `min` to `max` in `nseg` number of steps (default `nseg` is 15 for the AIS).

```python

def _ais_biophysics(self):
    """setup biophysics for ais. Channel distributions will be defined as gradients along segments."""
    self.ais.insert("na12")
    self.ais.insert("na16")
    self.ais.insert("kv")
    self.ais.insert("kd")
    na12_gradient = self.simple_ais_channel_gradient(self.gnav12_dist)
    na16_gradient = self.simple_ais_channel_gradient(self.gnav16_dist)
    for ind, seg in enumerate(self.ais):
        seg.gbar_na16 = na16_gradient[ind]
        seg.gbar_kd = 0
        seg.gbar_kv = self.ais_kv_gbar
    for ind, seg in enumerate(self.ais):
        seg.gbar_na12 = na12_gradient[ind]

```

# Myelin sheath

[Zbili et al. 2020](https://www.frontiersin.org/articles/10.3389/fncel.2020.00040/full) modeled the myelin membrane using the `extracellular` mechanism to add another layer of RC circuit to the internodes (Zbili et al. 2020 Sup. Fig 3):

<img src="img/sup_fig_3_zbili2020.jpg" alt="Zbili et al. 2020" width=600/>
Passive conductance of G<sub>my</sub> and C<sub>my</sub> depend on the number of myelinated wraps, and since each "wrap" is a bilayer, relative to the axon each sheath halves the G and C:

G<sub>my</sub>= G<sub>ax</sub>/2 and C<sub>my</sub>=C<sub>ax</sub>/2. 

Using a high magnification image from the lateral olfactory tract (LOT), I can count ~16 wraps spanning 173nm. 

<img src="img/HM_myelin_sheath.png" alt="LOT myelin sheath, unpublished data NMG" width=450/>


I implemented the myelin sheath based closely on the [published model](https://senselab.med.yale.edu/ModelDB/showmodel?model=263053&file=/zbili_debanne/myelinated_axon_1.hoc#tabs-2) ([Zbili et al. 2020](https://www.frontiersin.org/articles/10.3389/fncel.2020.00040/full)):

```python


def _myelinated_segments_biophysics(self):
    for myelin_section in self.myelinated_segs_list:
        myelin_section.insert("extracellular")
        myelin_section.xraxial[0] = 120663
        myelin_section.xraxial[1] = 10000000000
        myelin_section.xg[0] = 0.0000333 / (self.n_myelin_wraps * 2)
        myelin_section.xg[1] = 10000000000
        myelin_section.xc[0] = 1 / (self.n_myelin_wraps * 2)
        myelin_section.xc[1] = 0.0000000001
        myelin_section.e_extracellular = 0
        myelin_section.Ra = 150
        for myelin_seg in myelin_section:
            myelin_seg.cm = 0.012
            myelin_seg.g_pas = 1 / 100000

```

**Disclaimer:** From the [documentation](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html):

    Extracellular potentials do a great deal of violence to one’s intuition and it is important that the user carefully consider the results of simulations that use them. It is best to start out believing that there are bugs in the method and attempt to prove their existence.
**I have not done that.**

# Nodes of Ranvier

Nodes of Ranvier are composed of 3 segments (`nseg`) and have three types of channels: 

```python
def _node_biophysics(self):
    for node_section in self.nodes_list:
        node_section.insert("na16")
        node_section.insert("kd")
        node_section.insert("kv")
        node_section.Ra = 150
        node_section.cm = 1
        for node_seg in node_section:
            node_seg.g_pas = 0.0000333
            node_seg.gbar_na16 = self.node_gbar_na16
            node_seg.ena = 60
            node_seg.gbar_kd = self.node_gbar_kd
            node_seg.gbar_kv = self.node_gbar_kv
            node_seg.ek = -90
            node_seg.e_pas = -38.3
```

# How does AIS length affect AP firing properties in a Model of a mitral cell?